In [ ]:
import pandas as pd

In [ ]:
from rtoru import model as rt

exchanges = set(i.id for i in rt.exchanges)

In [ ]:
from concerto.helpers.biolog_to_exchange import biolog_map

In [ ]:
fname = 'Table_2_Multi-Omics Driven Metabolic Network Reconstruction and Analysis of Lignocellulosic Carbon Utilization in Rhodosporidium toruloides.XLSX'

bl_data = pd.read_excel(fname, sheet_name='Biolog')
bl_data['growth'] = bl_data['Average > 0.1 and all replicates greater than negative control']

In [ ]:
bl_data.PlateType.unique()

In [ ]:
biolog_map

In [ ]:
bl_data['plate'] = bl_data.PlateType + '-' + bl_data.Well
bl_data.growth = bl_data.growth.astype(bool)
bl_data.head()

In [ ]:
bl_data.PlateType.unique()

In [ ]:
bl_data.Experiment.unique()

In [ ]:
carbon_plate = bl_data.loc[bl_data.PlateType.isin(['PM1', 'PM2'])].copy()
nitrogren_plate = bl_data.loc[bl_data.PlateType.isin(['PM3B'])].copy()
nitrogren_plate.plate = nitrogren_plate.plate.str.replace('B', '')

phosh_plate =  bl_data.loc[bl_data.Experiment.isin(['Phosphorus'])].copy()
phosh_plate.plate = phosh_plate.plate.str.replace('A', '')

sulfer_plate =  bl_data.loc[bl_data.Experiment.isin(['Sulfur'])].copy()
sulfer_plate.plate = sulfer_plate.plate.str.replace('A', '')

In [ ]:
sulfer_plate.head(5)

In [ ]:
nitrogren_plate[['plate', 'Compound', 'growth']]

In [ ]:
biolog_map

In [ ]:
def merge_with_plate(plate):
    merged = pd.concat(
        [plate.set_index('plate'), biolog_map.set_index('plate')],
        axis=1,
        ignore_index=False
    )
    merged.dropna(subset=['growth', 'ex_rxn'], inplace=True)

    merged = merged.reset_index()
    merged#[['exchange', 'growth']]
    merged = subset_to_in_model(merged)[['exchange', 'growth']]
    merged['uptake'] = 10
    return merged

def subset_to_in_model(plate, model_ex=exchanges):
    return plate.loc[plate.exchange.isin(model_ex)]

carbon = merge_with_plate(carbon_plate)
nit = merge_with_plate(nitrogren_plate)
sulf = merge_with_plate(sulfer_plate)
phos = merge_with_plate(phosh_plate)

In [ ]:
sulf

In [ ]:
phos

In [ ]:
carbon.to_csv('../carbon_biolog.csv', index=False)
nit.to_csv('../nit_biolog.csv', index=False)
phos.to_csv('../phos_biolog.csv', index=False)
sulf.to_csv('../sulf_biolog.csv', index=False)